In [1]:
from get_response import PoliGPT
import rag_metrics as rm
import json

poligpt = PoliGPT(faiss_index_dir='../01_data/project_faiss')

FAISS inicializado - Vectores: 79044 | Dimensión: 768


In [2]:
def read_json(json_path):
    preguntas, respuestas_modelo = [], []
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for entry in data:
        for preg in entry["preguntas"]:
            preguntas.append(preg["pregunta"])
            respuestas_modelo.append(preg["respuesta"])

    return preguntas, respuestas_modelo

preguntas, respuestas_modelo = read_json("../01_data/preguntas.json")

In [3]:
respuestas = []
for pregunta in preguntas:  
    respuesta = poligpt.query_poligpt(pregunta, k_context=2)
    if respuesta['response'] != 'No dispongo de información suficiente en el contexto proporcionado.':
        respuestas.append(respuesta)

In [4]:
def extract_context_data(respuestas):
    """
    Toma la salida de PoliGPT (lista de dicts con 'context_used') y
    devuelve:
      - all_contexts: para cada respuesta, la lista de textos de contexto.
      - all_metadata: para cada respuesta, la lista de tuplas (doc_id, página).
    """
    all_contexts = []
    all_metadata = []
    for rsp in respuestas:
        ctxs = []
        metas = []
        for doc, score in rsp['context_used']:
            # doc es un objeto Document con atributos .metadata y .id
            ctxs.append(doc.page_content)
            # extraemos el id y la página:
            doc_id = getattr(doc, 'id', None)
            page   = doc.metadata.get('page')
            metas.append((doc_id, page))
        all_contexts.append(ctxs)
        all_metadata.append(metas)
    return all_contexts, all_metadata

contextos, metadatos = extract_context_data(respuestas)
respuestas_final = [dic['response'] for dic in respuestas]
preguntas_final = [dic['query'] for dic in respuestas]

In [5]:
def evaluate_dataset(preguntas, respuestas, contextos):
    """Aggregate the three metrics over an iterable of RAG records."""
    g_scores, f1s, qc_sims = [], [], []
    for answer, contexts, question in zip(respuestas, contextos,preguntas):
        g_scores.append(rm.grounding_score(answer, contexts))
        f1s.append(rm.context_overlap_f1(answer, contexts))
        qc_sims.append(rm.question_context_similarity(question, contexts))
    n = max(len(g_scores), 1)
    return {
        "Grounding": sum(g_scores) / n,
        "ContextOverlapF1": sum(f1s) / n,
        "QuestionContextSim": sum(qc_sims) / n,
    }

In [ ]:
def evaluate_with_references(preguntas, respuestas, contextos, preguntas_referencia):
    """Aggregate both reference-based and reference-free metrics.  
    Records must include keys:
      - "respuesta" (prediction)
      - "reference" (ground truth answer)
      - "pregunta", "contextos"
    """
    ems, f1_r, rouge_l, gs, f1s, sims = [], [], [], [], [], []
    for pred, ref, ctx, q in zip(respuestas, preguntas_referencia, contextos, preguntas):
        ems.append(rm.exact_match_score(pred, ref))
        f1_r.append(rm.token_f1_score(pred, ref))
        rouge_l.append(rm.rouge_l_score(pred, ref))
        gs.append(rm.grounding_score(pred, ctx))
        f1s.append(rm.context_overlap_f1(pred, ctx))
        sims.append(rm.question_context_similarity(q, ctx))
    n = max(len(ems), 1)
    return {
        "ExactMatch": sum(ems)/n,
        "TokenF1":    sum(f1_r)/n,
        "ROUGE_L":    sum(rouge_l)/n,
        "GroundingScore":  sum(gs)/n,
        "ContextOverlapF1":  sum(f1s)/n,
        "QuestionContextSim": sum(sims)/n
    }

In [7]:
evaluate_dataset(preguntas_final, respuestas_final, contextos)

{'Grounding': 0.585236022822446,
 'ContextOverlapF1': 0.3302083655276197,
 'QuestionContextSim': 0.14030019196411614}

In [8]:
evaluate_with_references(preguntas_final, respuestas_final, contextos, respuestas_modelo)

{'ExactMatch': 0.0,
 'TokenF1': 0.16386001907811462,
 'ROUGE_L': 0.13400927280945793,
 'Grounding': 0.585236022822446,
 'ContextF1': 0.3302083655276197,
 'Ques-ContextSim': 0.14030019196411614}